In [ ]:
import sys
!{sys.executable} -m pip install pip earthengine-api
!{sys.executable} -m pip install pip geemap

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

# Load data

In [ ]:
# read Aqueduct Floods data
floodImages = ee.ImageCollection("projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01/output_V06/inundation")
def constYear(img):
    img = ee.Image(img)  
    yearConst = 1980
    return ee.Algorithms.If(img.get('year_string'),img.set({'year':yearConst}),img.set({}))

floodImages = floodImages.map(constYear,False)

  # Shared flooding variables 
minInnundation = 0
    #note that return period 1.5 is only available for certain settings.
returnPeriodC = "rp0100"; # option [  "rp0002",  "rp0005",  "rp0010",  "rp0025", "rp0050",  "rp0100",  "rp0250",  "rp0500",  "rp1000"]
returnPeriodR = "rp00100"; # option [  "rp00002",  "rp00005",  "rp00010",  "rp00025", "rp00050",  "rp00100",  "rp00250",  "rp00500",  "rp01000"]
returnPeriodCstr = str(returnPeriodC)
#year = opts.get('year', 2030); # option [1980,2030,2050,2080] files that had year='hist' have a property called year_string since year is a reseverd keyword in earthengine.
startYear = 1980 #Options: 1980,2030,2050,2080.  If 1980, may need to remove some filters that are not available as properties in those images. 
endYear = 2050 #Options: 1980,2030,2050,2080.  If 1980, may need to remove some filters that are not available as properties in those images. 
startYearStr = str(startYear)
endYearStr = str(endYear)
climate = "rcp8p5"; # options "historical","rcp4p5", "rcp8p5"
  # Coastal flooding variables 
subsidence = 'nosub'; #options "nosub", "wtsub"
sea_level_rise_scenario = 50; # options 5, 50
  # Riverine flooding variables 
model = '0000GFDL-ESM2M';
  
Coastal_start = (floodImages.filterMetadata("floodtype","equals","inuncoast")
                      .filterMetadata("returnperiod","equals",returnPeriodC)
                      .filterMetadata("year","equals",startYear)
                      #.filterMetadata("climate","equals",climate)
                      #.filterMetadata("subsidence","equals",subsidence)
                      #.filterMetadata("sea_level_rise_scenario","equals",sea_level_rise_scenario)
                      .first()
                      #.unmask(ee.Image.constant(0));
                )
Coastal_end = (floodImages.filterMetadata("floodtype","equals","inuncoast")
                      .filterMetadata("returnperiod","equals",returnPeriodC)
                      .filterMetadata("year","equals",endYear)
                      .filterMetadata("climate","equals",climate)
                      .filterMetadata("subsidence","equals",subsidence)
                      .filterMetadata("sea_level_rise_scenario","equals",sea_level_rise_scenario)
                      .first()
                      #.unmask(ee.Image.constant(0));
                  )
imageVisParam = {"opacity":1,"bands":["b1"],"max":2,"palette":["ffffff","00cfff","0013ff"]};
#Map.addLayer(Coastal_end,imageVisParam,"Coastal Flooding",1,0.7)


Riverine_start = (floodImages.filterMetadata("floodtype","equals","inunriver")
                      .filterMetadata("returnperiod","equals",returnPeriodR)
                      .filterMetadata("year","equals",startYear)
                      #.filterMetadata("climate","equals",climate)
                      #.filterMetadata("model", "equals", model)
                      .first()
                      #.unmask(ee.Image.constant(0));
                 )
Riverine_end = (floodImages.filterMetadata("floodtype","equals","inunriver")
                      .filterMetadata("returnperiod","equals",returnPeriodR)
                      .filterMetadata("year","equals",endYear)
                      .filterMetadata("climate","equals",climate)
                      .filterMetadata("model", "equals", model)                      
                      .first()
                      #.unmask(ee.Image.constant(0));
               )
imageVisParam = {"opacity":1,"bands":["b1"],"max":2,"palette":["ffffff","00cfff","0013ff"]};
#Map.addLayer(Riverine_end,imageVisParam,"Riverine Flooding",1,0.7);

Combflood_start = Coastal_start.max(Riverine_start)
Combflood_start = Combflood_start.updateMask(Combflood_start.gt(minInnundation))
Combflood_end = Coastal_end.max(Riverine_end)
Combflood_end = Combflood_end.updateMask(Combflood_end.gt(minInnundation))
Combflood_endScale = Combflood_end.projection().nominalScale()
# print(Combflood_end)
# Map.addLayer(Combflood_end,imageVisParam,"Combined Flooding",1,0.7);


In [ ]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

# Extract flood areas by city

In [ ]:
for i in range(0,1):#len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # Download ee.Image of albedo as GeoTIFF to Google Drive 
    geemap.ee_export_image_to_drive(
        Combflood_end.toFloat(), # use toFloat() to reduce file size
        description = boundary_id + '-flood-innundation-'+endYearStr+'-'+returnPeriodCstr+'',
        folder='data', 
        scale=Combflood_endScale, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

# specify bucket name
bucket_name = 'cities-cities4forests' 

In [ ]:
out_dir = os.getcwd()

In [ ]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id + '-flood-innundation-'+endYearStr+'-'+returnPeriodCstr+'.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/flooding/aqueduct/v_0/' + boundary_id + '-flood-innundation-'+endYearStr+'-'+returnPeriodCstr+'.tif',
                               ExtraArgs={'ACL':'public-read'})

# Visualize data

In [ ]:
## create map
Map = geemap.Map(height="350px")
Map

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(boundary_geo_ee, zoom=8)

In [ ]:
Map.addLayer(greenmask),
             {'min':0, 'max':0.5, 'palette':['white','#006400']},
             'Vegetation cover',True,1)